### **1.   Lire le fichier "X_train_uptade.csv" et "Y_trainCVw08PX.csv"**

In [1]:
# Importation des librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Chargement des fichiers '="X_train_uptade.csv" et "Y_trainCVw08PX.csv"
X_train = pd.read_csv('../../X_train.csv', index_col=0)
y_train = pd.read_csv('../../Y_train.csv', index_col=0)

# Afficher le nombre de lignes et de colonnes de chacun des deux fichiers
print("Le nombre de lignes et de colonnes de X_train est de :",X_train.shape,
      "\nLe nombre de lignes et de colonnes de y_train est de :", y_train.shape)

Le nombre de lignes et de colonnes de X_train est de : (84916, 4) 
Le nombre de lignes et de colonnes de y_train est de : (84916, 1)


### **2.   Fusion des fichiers "X_train_uptade.csv" et "Y_trainCVw08PX.csv"**




In [2]:
# Fusion avec merge des deux datasets
df = pd.merge(X_train, y_train, left_index=True, right_index=True)

### **3.   Afficher un aperçu des 10 premières lignes**






In [3]:
# Affichage des 5 premières lignes
display(df.head())

# Affichage des 5 dernières lignes
display(df.tail())


,designation,description,productid,imageid,prdtypecode
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,1280
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,2705


,designation,description,productid,imageid,prdtypecode
84911,The Sims [ Import Anglais ],NaN,206719094,941495734,40
84912,Kit piscine acier NEVADA déco pierre Ø 3.50m x...,<b>Description complète :</b><br />Kit piscine...,3065095706,1188462883,2583
84913,Journal Officiel De La Republique Francaise N°...,NaN,440707564,1009325617,2280
84914,Table Basse Bois De Récupération Massif Base B...,<p>Cette table basse a un design unique et con...,3942400296,1267353403,1560
84915,Gomme De Collection 2 Gommes Pinguin Glace Ver...,NaN,57203227,684671297,2522


### **4.   Fusion des colonnes Description et Designation**

In [4]:
# Créer une nouvelle colonne "description_complete" en concaténant les valeurs de "description" et "designation"
df['description_complete'] = df['designation'].fillna('') + ' ' + df['description'].fillna('')
df.drop(['description', 'designation'], axis=1, inplace=True)
df.head()

,productid,imageid,prdtypecode,description_complete
0,3804725264,1263597046,10,Olivia: Personalisiertes Notizbuch / 150 Seite...
1,436067568,1008141237,2280,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...
2,201115110,938777978,50,Grand Stylet Ergonomique Bleu Gamepad Nintendo...
3,50418756,457047496,1280,Peluche Donald - Europe - Disneyland 2000 (Mar...
4,278535884,1077757786,2705,La Guerre Des Tuques Luc a des id&eacute;es de...


### **5.   cleaning de description_complete**



In [5]:
from bs4 import BeautifulSoup


def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df['description_complete'] = df['description_complete'].apply(lambda x: remove_html_tags(x) if pd.notnull(x) else x)

df.head()


c:\Users\tgp\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,productid,imageid,prdtypecode,description_complete
0,3804725264,1263597046,10,Olivia: Personalisiertes Notizbuch / 150 Seite...
1,436067568,1008141237,2280,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...
2,201115110,938777978,50,Grand Stylet Ergonomique Bleu Gamepad Nintendo...
3,50418756,457047496,1280,Peluche Donald - Europe - Disneyland 2000 (Mar...
4,278535884,1077757786,2705,La Guerre Des Tuques Luc a des idées de grande...


In [6]:
# Importation de la librairie worcloud et de la fonction WordCloud
from wordcloud import WordCloud

# Suggestions de la gestion des erreurs de Python (Google Colab)
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

# Fonction pour le prétraitement du texte
def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les caractères spéciaux et les chiffres
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenisation
    words = word_tokenize(text)
    # Créer une liste de stopwords en français
    stopwords_french = stopwords.words('french')
    # Ajouter "X" et "x" à la liste de stopwords
    stopwords_french.extend(["Ap.", "Apr.", "GHz", "MHz", "USD", "a", "afin", "ah", "ai", "aie", "aient", "aies", "ait", "alors", "après", "as", "attendu",
                             "au", "au-delà", "au-devant", "aucun", "aucune", "audit", "auprès", "auquel", "aura", "aurai", "auraient", "aurais", "aurait",
                             "auras", "aurez", "auriez", "aurions", "aurons", "auront", "aussi", "autour", "autre", "autres", "autrui", "aux", "auxdites", "auxdits",
                             "auxquelles", "auxquels", "avaient", "avais", "avait", "avant", "avec", "avez", "aviez", "avions", "avons", "ayant", "ayez", "ayons", "b",
                             "bah", "banco", "ben", "bien", "bé", "c", "c'", "c'est", "c'était", "car", "ce", "ceci", "cela", "celle", "celle-ci", "celle-là", "celles",
                             "celles-ci", "celles-là", "celui", "celui-ci", "celui-là", "celà", "cent", "cents", "cependant", "certain", "certaine", "certaines", "certains",
                             "ces", "cet", "cette", "ceux", "ceux-ci", "ceux-là", "cf.", "cg", "cgr", "chacun", "chacune", "chaque", "chez", "ci", "cinq", "cinquante", "cinquante-cinq",
                             "cinquante-deux", "cinquante-et-un", "cinquante-huit", "cinquante-neuf", "cinquante-quatre", "cinquante-sept", "cinquante-six", "cinquante-trois", "cl", "cm",
                             "cm²", "comme", "contre", "d", "d'", "d'après", "d'un", "d'une", "dans", "de", "depuis", "derrière", "des", "desdites", "desdits", "desquelles", "desquels",
                             "deux", "devant", "devers", "dg", "différentes", "différents", "divers", "diverses", "dix", "dix-huit", "dix-neuf", "dix-sept", "dl", "dm", "donc", "dont",
                             "douze", "du", "dudit", "duquel", "durant", "dès", "déjà", "e", "eh", "elle", "elles", "en", "en-dehors", "encore", "enfin", "entre", "envers", "es", "est",
                             "et", "eu", "eue", "eues", "euh", "eurent", "eus", "eusse", "eussent", "eusses", "eussiez", "eussions", "eut", "eux", "eûmes", "eût", "eûtes", "f", "fait", "fi",
                             "flac", "fors", "furent", "fus", "fusse", "fussent", "fusses", "fussiez", "fussions", "fut", "fûmes", "fût", "fûtes", "g", "gr", "h", "ha", "han", "hein", "hem",
                             "heu", "hg", "hl", "hm", "hm³", "holà", "hop", "hormis", "hors", "huit", "hum", "hé", "i", "ici", "il", "ils", "j", "j'", "j'ai", "j'avais", "j'étais", "jamais",
                             "je", "jusqu'", "jusqu'au", "jusqu'aux", "jusqu'à", "jusque", "k", "kg", "km", "km²", "l", "l'", "l'autre", "l'on", "l'un", "l'une", "la", "laquelle", "le", "lequel",
                             "les", "lesquelles", "lesquels", "leur", "leurs", "lez", "lors", "lorsqu'", "lorsque", "lui", "lès", "m", "m'", "ma", "maint", "mainte", "maintes", "maints", "mais",
                             "malgré", "me", "mes", "mg", "mgr", "mil", "mille", "milliards", "millions", "ml", "mm", "mm²", "moi", "moins", "mon", "moyennant", "mt", "m²", "m³", "même", "mêmes",
                             "n", "n'avait", "n'y", "ne", "neuf", "ni", "non", "nonante", "nonobstant", "nos", "notre", "nous", "nul", "nulle", "nº", "néanmoins", "o", "octante", "oh", "on", "ont",
                             "onze", "or", "ou", "outre", "où", "p", "par", "par-delà", "parbleu", "parce", "parmi", "pas", "passé", "pendant", "personne", "peu", "plus", "plus_d'un", "plus_d'une",
                             "plusieurs", "pour", "pourquoi", "pourtant", "pourvu", "près", "puisqu'", "puisque", "q", "qu", "qu'", "qu'elle", "qu'elles", "qu'il", "qu'ils", "qu'on", "quand", "quant",
                             "quarante", "quarante-cinq", "quarante-deux", "quarante-et-un", "quarante-huit", "quarante-neuf", "quarante-quatre", "quarante-sept", "quarante-six", "quarante-trois", "quatorze",
                             "quatre", "quatre-vingt", "quatre-vingt-cinq", "quatre-vingt-deux", "quatre-vingt-dix", "quatre-vingt-dix-huit", "quatre-vingt-dix-neuf", "quatre-vingt-dix-sept", "quatre-vingt-douze",
                             "quatre-vingt-huit", "quatre-vingt-neuf", "quatre-vingt-onze", "quatre-vingt-quatorze", "quatre-vingt-quatre", "quatre-vingt-quinze", "quatre-vingt-seize", "quatre-vingt-sept",
                             "quatre-vingt-six", "quatre-vingt-treize", "quatre-vingt-trois", "quatre-vingt-un", "quatre-vingt-une", "quatre-vingts", "que", "quel", "quelle", "quelles", "quelqu'", "quelqu'un",
                             "quelqu'une", "quelque", "quelques", "quelques-unes", "quelques-uns", "quels", "qui", "quiconque", "quinze", "quoi", "quoiqu'", "quoique", "r", "revoici", "revoilà", "rien", "s", "s'",
                             "sa", "sans", "sauf", "se", "seize", "selon", "sept", "septante", "sera", "serai", "seraient", "serais", "serait", "seras", "serez", "seriez", "serions", "serons", "seront", "ses", "si",
                             "sinon", "six", "soi", "soient", "sois", "soit", "soixante", "soixante-cinq", "soixante-deux", "soixante-dix", "soixante-dix-huit", "soixante-dix-neuf", "soixante-dix-sept", "soixante-douze",
                             "soixante-et-onze", "soixante-et-un", "soixante-et-une", "soixante-huit", "soixante-neuf", "soixante-quatorze", "soixante-quatre", "soixante-quinze", "soixante-seize", "soixante-sept",
                             "soixante-six", "soixante-treize", "soixante-trois", "sommes", "son", "sont", "sous", "soyez", "soyons", "suis", "suite", "sur", "sus", "t", "t'", "ta", "tacatac", "tandis", "te", "tel",
                             "telle", "telles", "tels", "tes", "toi", "ton", "toujours", "tous", "tout", "toute", "toutefois", "toutes", "treize", "trente", "trente-cinq", "trente-deux", "trente-et-un", "trente-huit",
                             "trente-neuf", "trente-quatre", "trente-sept", "trente-six", "trente-trois", "trois", "très", "tu", "u", "un", "une", "unes", "uns", "v", "vers", "via", "vingt", "vingt-cinq", "vingt-deux",
                             "vingt-huit", "vingt-neuf", "vingt-quatre", "vingt-sept", "vingt-six", "vingt-trois", "vis-à-vis", "voici", "voilà", "vos", "votre", "vous", "w", "x", "y", "z", "zéro", "à", "ç'", "ça", "ès",
                             "étaient", "étais", "était", "étant", "étiez", "étions", "été", "étée", "étées", "étés", "êtes", "être", "ô"])
    # Supprimer les stopwords
    words = [word for word in words if word not in stopwords_french]
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Prétraiter la colonne description_complete
df['clean_description_complete'] = df['description_complete'].apply(preprocess_text)

df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tgp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tgp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tgp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,productid,imageid,prdtypecode,description_complete,clean_description_complete
0,3804725264,1263597046,10,Olivia: Personalisiertes Notizbuch / 150 Seite...,olivia personalisiertes notizbuch seiten punkt...
1,436067568,1008141237,2280,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,journal art lart marche salon dart asiatique p...
2,201115110,938777978,50,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,grand stylet ergonomique bleu gamepad nintendo...
3,50418756,457047496,1280,Peluche Donald - Europe - Disneyland 2000 (Mar...,peluche donald europe disneyland marionnette d...
4,278535884,1077757786,2705,La Guerre Des Tuques Luc a des idées de grande...,guerre tuques luc ides grandeur veut organiser...


#### **6 Traduction**
La traduction prend trop de temps pour être utilisable.

In [7]:
from mtranslate import translate

# Supposons que votre dataframe s'appelle df

# Fonction pour traduire le texte en anglais
def translate_to_english(text):
    return translate(text, "en")

# Appliquer la traduction à la colonne 'description_complete'
df['description_complete_english_2'] = df['clean_description_complete'].apply(lambda x: translate_to_english(x) if pd.notnull(x) else x)

# Afficher le nouveau dataframe
print(df.head())

KeyboardInterrupt: 

In [ ]:
from googletrans import Translator

# Fonction pour traduire le texte en anglais
def translate_to_english(text):
    translator = Translator()
    try:
        translated_text = translator.translate(text, dest='en').text
        return translated_text
    except Exception as e:
        print(f"Erreur de traduction : {e}")
        return text

# Appliquer la traduction à la colonne 'clean_description_complete'
df['description_complete_english'] = df['clean_description_complete'].apply(lambda x: translate_to_english(x) if pd.notnull(x) else x)

# Afficher le nouveau dataframe
print(df.head())

Erreur de traduction : the JSON object must be str, bytes or bytearray, not NoneType
Erreur de traduction : The read operation timed out
Erreur de traduction : The read operation timed out
Erreur de traduction : The read operation timed out
Erreur de traduction : The read operation timed out
Erreur de traduction : The read operation timed out
Erreur de traduction : the JSON object must be str, bytes or bytearray, not NoneType
Erreur de traduction : The read operation timed out
Erreur de traduction : the JSON object must be str, bytes or bytearray, not NoneType
Erreur de traduction : The read operation timed out
Erreur de traduction : The read operation timed out
Erreur de traduction : The read operation timed out
Erreur de traduction : The read operation timed out
Erreur de traduction : The read operation timed out
Erreur de traduction : The read operation timed out
Erreur de traduction : The read operation timed out
Erreur de traduction : The read operation timed out
Erreur de traduct

KeyboardInterrupt: 